In [1]:
""" 
    深度学习框架会在后端自动构建计算图，利用计算图，系统可以了解所有依赖关系，
并且选择性地并行执行多个不相互依赖的任务提高速度 
    通常情况下，单个操作符将使用所有CPU或单个GPU上所有计算资源。如，即时在一台
机器上有多个CPU处理器，dot操作符也将使用所有核心(和线程)。这样的行为同样适用于
单个GPU。因此并行化对单设备计算机不是很有用，而常用于多GPU之间
    预热设备确保缓存的作用不影响最终的结果--对设备执行一次传递
    torch.cuda.synchronize()会等待一个CUDA设备的所有流中的所有核心的计算完成，
函数接受一个device参数，代表哪个设备需要同步。如果device=None，将使用current_device()找出当前设备
    如果删除两任务之间的synchronize语句，系统就可以在两个设备上自动实现并行计算。
"""

' \n    深度学习框架会在后端自动构建计算图，利用计算图，系统可以了解所有依赖关系，\n并且选择性地并行执行多个不相互依赖的任务提高速度 \n    通常情况下，单个操作符将使用所有CPU或单个GPU上所有计算资源。如，即时在一台\n机器上有多个CPU处理器，dot操作符也将使用所有核心(和线程)。这样的行为同样适用于\n单个GPU。因此并行化对单设备计算机不是很有用，而常用于多GPU之间\n    预热设备确保缓存的作用不影响最终的结果--对设备执行一次传递\n    torch.cuda.synchronize()会等待一个CUDA设备的所有流中的所有核心的计算完成，\n函数接受一个device参数，代表哪个设备需要同步。如果device=None，将使用current_device()找出当前设备\n    如果删除两任务之间的synchronize语句，系统就可以在两个设备上自动实现并行计算。\n'

In [2]:
import torch
from d2l import torch as d2l

devices = d2l.try_all_gpus()

def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.randn(size=(400, 400), device=devices[0])
x_gpu2 = torch.randn(size=(400, 400), device=devices[0])

In [3]:
run(x_gpu1)
torch.cuda.synchronize(device=devices[0])

In [4]:
with d2l.Benchmark('GPU0 time'):
    run(x_gpu1)
    torch.cuda.synchronize(device=devices[0])
with d2l.Benchmark('GPU0 time'):
    run(x_gpu2)
    torch.cuda.synchronize(device=devices[0])

GPU0 time: 0.0140 sec
GPU0 time: 0.0130 sec


In [5]:
with d2l.Benchmark('GPU0 time'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize(device=devices[0])

GPU0 time: 0.0260 sec


In [8]:
""" 并行计算与通信 """
""" 在不同设备之间移动数据，CPU&GPU GPU与GPU """
"""
Dataloader()加载数据时参数pin_memory=True为生成的Tensor放在锁页
Tensor.to or .cuda()参数non_blocking=True意为只放入GPU而不取出
"""
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('在gpu0上运行'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('复制到CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

在gpu0上运行: 0.0140 sec
复制到CPU: 0.0270 sec


In [9]:
with d2l.Benchmark('在GPU0上运行并复制到CPU'):
    y = run(x_gpu1)
    copy_to_cpu(y, non_blocking=True)
    torch.cuda.synchronize()

在GPU0上运行并复制到CPU: 0.0170 sec
